In [ ]:
%run "dslabs_functions.py"

In [ ]:
import pandas as pd
import numpy as np


filepath=r'df_merch_pre_proc.csv'

file_tag = "df_merch_pre_proc"


# test_data=True
test_data=False

if test_data==True:

    data=pd.read_csv(filepath)

    # 10% sample
    data=data.sample(frac=0.1, replace=False)
    

else:
    data=pd.read_csv(filepath)


data.info()

### Symbolic Distribution filtered to top 10 value counts dimensions' per variable

In [ ]:

# Assuming you have already created a 'returning_user' column in your data

symbolic: list[str] = variable_types["symbolic"] + variable_types["binary"]

# Check if there are symbolic variables
if [] != symbolic:
    rows, cols = define_grid(len(symbolic))
    
    # Increase the figure size to improve spacing (adjust the scale as needed)
    fig, axs = subplots(
        rows, cols, figsize=(cols * HEIGHT * 1.5, rows * HEIGHT * 1.5), squeeze=False
    )
    
    i, j = 0, 0
    for n in range(len(symbolic)):
        # Get value counts for the symbolic variable and limit to top 10 categories
        top_10_categories = data[symbolic[n]].value_counts().nlargest(10).index

        # Filter the data to include only the top 10 categories for the symbolic variable
        filtered_data = data[data[symbolic[n]].isin(top_10_categories)]

        # Group by the symbolic variable and 'returning_user', and get counts
        grouped_counts: pd.DataFrame = filtered_data.groupby([symbolic[n], 'returning_user']).size().unstack(fill_value=0)

        # Get the index (top 10 categories) and values (counts of new and returning users)
        categories = grouped_counts.index.to_list()
        new_users = grouped_counts[0].to_list()  # New users (ga_session_number = 1)
        returning_users = grouped_counts[1].to_list()  # Returning users (ga_session_number > 1)

        # Plot stacked bar chart with adjusted bar width
        bar_width = 0.6  # Adjust bar width
        axs[i, j].bar(categories, new_users, width=bar_width, label="New Users", color="lightblue")
        axs[i, j].bar(categories, returning_users, bottom=new_users, width=bar_width, label="Returning Users", color="orange")

        # Add chart details
        axs[i, j].set_title(f"Stacked Histogram for {symbolic[n]} (Top 10)")
        axs[i, j].set_xlabel(symbolic[n])
        axs[i, j].set_ylabel("nr records")
        
        # Rotate x-axis labels for better readability
        axs[i, j].tick_params(axis='x', rotation=45)

        # Add legend
        axs[i, j].legend()

        # Move to the next subplot
        i, j = (i + 1, 0) if (n + 1) % cols == 0 else (i, j + 1)

    # Adjust layout for better fitting
    fig.tight_layout()

    # Show the plot
    show()

else:
    print("There are no symbolic variables.")